In [1]:
#Standaard imports
import pickle

import numpy as np
import pandas as pd
from pandas_summary import DataFrameSummary
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm import tqdm
from feature_engine.encoding import OneHotEncoder
from sklearn import set_config
from sklearn.base import clone
from sklearn.ensemble import BaggingRegressor
from sklearn.preprocessing import MinMaxScaler
from feature_engine.encoding import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from skmultilearn.problem_transform import LabelPowerset
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier

C:\Users\Alex van Tol\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv("origen.csv")
df = df.drop('Unnamed: 0', axis=1) ##We get rid of the first column which only serves to enumerate data

In [4]:
df = df.dropna(subset=['genres_x']) ##We are going to use genres_x later on but we remove the 2 NaN entry rows.

In [5]:
avg_rating = df['averageRating']
df = df.drop('averageRating', axis = 1)

In [5]:
df.shape

(977541, 28)

In [7]:
#Calculate new column of data consisting of amount of years running of a show
df['endYear'] = df['endYear'].replace(0, 2023)
df['Years'] = df['endYear'] - df['startYear']
df = df.drop(columns=['startYear', 'endYear'])

In [8]:
df.head()

,numVotes,titleType,isAdult,runtimeMinutes,genres_x,directors,writers,seasonNumber,episodeNumber,ordering,...,original_language,popularity,production_companies,production_countries,revenue,runtime,status,tagline,video,Years
0,15,movie,0.0,91,"Comedy,Musical",nm0883334,"nm0844784,nm0305863",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72
1,990,tvSeries,0.0,30,"Action,Adventure,Animation","nm2291816,nm3088555,nm4930005,nm1746040","nm1707665,nm0789712,nm0403945,nm1826186,nm0630...",NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14
2,41,tvEpisode,0.0,44,"Documentary,History,War",nm0414025,"nm0414025,nm3692091,nm1620376",1.0,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12
3,48,movie,0.0,84,Drama,nm2977268,"nm2977268,nm0415515",NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54
4,28,movie,0.0,130,"Comedy,Drama",nm2366663,"nm4290500,nm4289029",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13


In [9]:
discrete_columns = []
numerical_columns = [c for c in df.columns if df[c].dtype != 'O' and c not in discrete_columns]
categorical_columns = [c for c in df.columns if df[c].dtype == 'O']

In [10]:
df_num = df[numerical_columns]
# 0   numVotes         977541 non-null  int64
# 1   isAdult          977541 non-null  float64
# 2   startYear        977541 non-null  int64
# 3   endYear          977541 non-null  int64
# 4   runtimeMinutes   977541 non-null  int64
# 5   seasonNumber     438243 non-null  float64
# 6   episodeNumber    438243 non-null  float64
# 7   ordering         370623 non-null  float64
# 8   isOriginalTitle  370623 non-null  float64
# 9   budget           47370 non-null   float64
# 10  popularity       47369 non-null   float64
# 11  revenue          47369 non-null   float64
# 12  runtime          47158 non-null   float64

In [11]:
df_num.columns

Index(['numVotes', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes',
       'seasonNumber', 'episodeNumber', 'ordering', 'isOriginalTitle',
       'budget', 'popularity', 'revenue', 'runtime'],
      dtype='object')

In [12]:
df_cat = df[categorical_columns]
#titleType             977541 non-null  object
# 1   genres_x              977539 non-null  object
# 2   directors             977541 non-null  object
# 3   writers               977541 non-null  object
# 4   language              370623 non-null  object
# 5   attributes            370623 non-null  object
# 6   adult                 47370 non-null   object
# 7   genres_y              47370 non-null   object
# 8   original_language     47358 non-null   object
# 9   production_companies  47369 non-null   object
# 10  production_countries  47369 non-null   object
# 11  status                47299 non-null   object
# 12  tagline               23845 non-null   object
# 13  video                 47369 non-null   object

In [13]:
##Data pre-processing
def PreProc(df_num):
    df_num.loc[:, 'episodeNumber'] = df_num.loc[:, 'episodeNumber'] .fillna(1)
    df_num.loc[df_num.loc[:, 'episodeNumber']  == 0, 'episodeNumber'] = 1 #Every  episode that is zero or NaN is 1 --> atleast one chapter for a program
    df_num.loc[:, 'seasonNumber'] = df_num.loc[:, 'seasonNumber'].fillna(0) #Every Nan season number gets the value 0
    return df_num

In [14]:
df_num = PreProc(df_num)

C:\Users\Alex van Tol\anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\Alex van Tol\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [15]:
col_idx =['directors', 'writers']
df_cat[col_idx].head()

,directors,writers
0,nm0883334,"nm0844784,nm0305863"
1,"nm2291816,nm3088555,nm4930005,nm1746040","nm1707665,nm0789712,nm0403945,nm1826186,nm0630..."
2,nm0414025,"nm0414025,nm3692091,nm1620376"
3,nm2977268,"nm2977268,nm0415515"
4,nm2366663,"nm4290500,nm4289029"


In [16]:
col_idx = ['directors', 'writers']
for col in col_idx:
    df_cat[col] = df_cat[col].str.lstrip('nm')
    df_cat[col] = df_cat[col].str.split(',').str[0]

C:\Users\ALEXVA~1\AppData\Local\Temp/ipykernel_16968/3727868945.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat[col] = df_cat[col].str.lstrip('nm')
C:\Users\ALEXVA~1\AppData\Local\Temp/ipykernel_16968/3727868945.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat[col] = df_cat[col].str.split(',').str[0]


In [17]:
df_cat[col_idx].head()

,directors,writers
0,0883334,0844784
1,2291816,1707665
2,0414025,0414025
3,2977268,2977268
4,2366663,4290500


In [19]:
nan_count = df_cat['genres_x'].isnull().sum()
print("Number of NaN values in 'genres_x' column:", nan_count)

Number of NaN values in 'genres_x' column: 0


In [20]:
df_cat = pd.get_dummies(df_cat, columns=['titleType'], prefix=['titleType'])

In [22]:
df_cat['genres_x'] = df_cat['genres_x'].str.split(',')
df_cat = pd.concat([df_cat, df_cat['genres_x'].apply(lambda x: pd.Series([1] * len(x), index=x)).fillna(0)], axis=1)

In [23]:
df_cat.drop('genres_x', axis=1, inplace=True)

In [24]:
df_cat.head()

,directors,writers,language,attributes,adult,genres_y,original_language,production_companies,production_countries,status,...,Sport,Game-Show,News,Fantasy,Family,Reality-TV,0,Adult,Thriller,Film-Noir
0,0883334,0844784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2291816,1707665,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0414025,0414025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2977268,2977268,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2366663,4290500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
num_6_col = ['numVotes','isAdult', 'Years', 'runtimeMinutes','seasonNumber','episodeNumber']

In [27]:
df_num_6 = df_num[num_6_col]

In [28]:
df_num_6.head()

,numVotes,isAdult,Years,runtimeMinutes,seasonNumber,episodeNumber
0,15,0.0,72,91,0.0,1.0
1,990,0.0,14,30,0.0,1.0
2,41,0.0,12,44,1.0,5.0
3,48,0.0,54,84,0.0,1.0
4,28,0.0,13,130,0.0,1.0


In [29]:
columns_to_exclude = ['language', 'adult', 'genres_y', 'original_language', 'production_companies', 'production_countries','attributes', 'status','tagline','video']
df_cat_fin = df_cat.drop(columns=columns_to_exclude)

In [30]:
df_cat_fin.head()

,directors,writers,titleType_movie,titleType_short,titleType_tvEpisode,titleType_tvMiniSeries,titleType_tvMovie,titleType_tvSeries,titleType_tvShort,titleType_tvSpecial,...,Sport,Game-Show,News,Fantasy,Family,Reality-TV,0,Adult,Thriller,Film-Noir
0,0883334,0844784,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2291816,1707665,0,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0414025,0414025,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2977268,2977268,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2366663,4290500,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
final = pd.concat([df_num_6,df_cat_fin], axis=1)

In [32]:
final.shape

(977539, 47)

In [34]:
column_list = ['numVotes', 'isAdult', 'Years', 'runtimeMinutes', 'seasonNumber',
       'episodeNumber', 'directors', 'writers', 'titleType_movie',
       'titleType_short', 'titleType_tvEpisode', 'titleType_tvMiniSeries',
       'titleType_tvMovie', 'titleType_tvSeries', 'titleType_tvShort',
       'titleType_tvSpecial', 'titleType_video', 'titleType_videoGame',
       'Comedy', 'Musical', 'Action', 'Adventure', 'Animation', 'Documentary',
       'History', 'War', 'Drama', 'Mystery', 'Romance', 'Short', 'Music',
       'Talk-Show', 'Crime', 'Horror', 'Biography', 'Western', 'Sci-Fi',
       'Sport', 'Game-Show', 'News', 'Fantasy', 'Family', 'Reality-TV', '0',
       'Adult', 'Thriller', 'Film-Noir']
#To make sure the entries are the same in the train and test notebooks

In [37]:
final = final.reindex(columns=column_list)

In [40]:
final.head()

,numVotes,isAdult,Years,runtimeMinutes,seasonNumber,episodeNumber,directors,writers,titleType_movie,titleType_short,...,Sport,Game-Show,News,Fantasy,Family,Reality-TV,0,Adult,Thriller,Film-Noir
0,15,0.0,72,91,0.0,1.0,0883334,0844784,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,990,0.0,14,30,0.0,1.0,2291816,1707665,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,41,0.0,12,44,1.0,5.0,0414025,0414025,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,48,0.0,54,84,0.0,1.0,2977268,2977268,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,28,0.0,13,130,0.0,1.0,2366663,4290500,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
RF = RandomForestRegressor(n_estimators=100, verbose=True, max_depth=50, n_jobs=-1, min_samples_split=3)


In [42]:
RFfit = RF.fit(final,avg_rating)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  9.5min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 15.7min finished


In [ ]:
with open('saved_model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [46]:
submission = pd.DataFrame(predictions, columns=["averageRating"])

In [47]:
submission.index.name = "Id"

In [49]:
submission.to_csv("submission.csv")